In [ ]:
import spark
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import LDA

df = spark.createDataFrame([[1, Vectors.dense([0.0, 1.0])],
                                [2, Vectors.dense([2.0, 3.0])],],
                                ["id", "features"])

lda = LDA(k=2, seed=1)
lda.setMaxIter(10)

model = lda.fit(df)

model.vocabSize()

model.describeTopics().show()

In [ ]:
import pyspark

from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").\
                            config("spark.driver.maxResultSize", "0").\
                            config("spark.kryoserializer.buffer.max", "2000M").\
                            config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.0.2").\
                            appName('chapter_6').\
                            getOrCreate()

# Getting the Data